In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
import pymysql

In [2]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [3]:
# mysql 연동
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'finaldb'
                , charset='utf8'
                ) 

cur = conn.cursor()

In [7]:
# input
new_title = "크리스마스에 듣기 좋은 신나는 노래"

In [8]:
# 플리 id 마지막 값 가져오기
cur = conn.cursor()
cur.execute('select ply_id from ply_meta order by ply_id desc limit 1;')
last_ply_id = cur.fetchall()
new_ply_id = last_ply_id[0][0] + 1

In [6]:
# Insert new ply_title 
# cur.execute(f'INSERT INTO ply_meta (ply_id, ply_title, ply_updt) VALUES ({new_ply_id}, "{new_title}", now());')
# conn.commit()

In [9]:
# input_title embedding 값 만들기
new_embedding = model.encode(new_title)

In [8]:
# Insert new title embedding
# cur.execute(f'INSERT INTO ply_title_embedding (ply_id, ply_title_emd) VALUES ({new_ply_id}, "{new_embedding}");')
# conn.commit()

In [11]:
# 가장 유사한 플레이리스트 추출
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [23]:
sql = 'SELECT * FROM ply_title_embedding'
ply_title_embedding = pd.read_sql(sql, conn)

# str -> embedding(array)
ply_title_embedding['ply_title_emd'] = ply_title_embedding.apply(lambda x: np.fromstring(x['ply_title_emd'].strip('[]'),dtype=float, sep=' '), axis=1)

# cosine similarity 계산
ply_title_embedding['cos_sim'] = ply_title_embedding.apply(lambda x: cos_sim(x['ply_title_emd'], new_embedding), axis=1)

# 유사한 플리 2개 id 추출
sim_ply_id = list(ply_title_embedding.sort_values(by='cos_sim', ascending=False)['ply_id'][:2])

C:\Users\k9942\AppData\Local\Temp\ipykernel_15612\645151206.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ply_title_embedding = pd.read_sql(sql, conn)


In [24]:
sim_ply_id

[2, 0]

In [7]:
conn.commit()

In [8]:
conn.close()